In [39]:
import pandas as pd
import numpy as np
from dash import dcc, html, Input, Output, dash_table, Dash
import plotly.express as px

SyntaxError: invalid syntax (1546532453.py, line 1)

In [17]:
file_path = "C:/Users/ConnorChristensen/OneDrive - Wyoming Business Council/Documents/GitHub/App_Econ_Div/df_around_wy.csv"  
df_around_wy = pd.read_csv(file_path)

In [18]:
print(df_around_wy.head(25))

             Area Name                                      Industry Name  \
0   Larimer County, CO                            Timber Tract Operations   
1   Larimer County, CO  Forest Nurseries and Gathering of Forest Products   
2   Larimer County, CO                                            Logging   
3   Larimer County, CO                                    Finfish Fishing   
4   Larimer County, CO                                  Shellfish Fishing   
5   Larimer County, CO                               Other Marine Fishing   
6   Larimer County, CO                               Hunting and Trapping   
7   Larimer County, CO                                     Cotton Ginning   
8   Larimer County, CO              Crop Harvesting, Primarily by Machine   
9   Larimer County, CO  Postharvest Crop Activities (except Cotton Gin...   
10  Larimer County, CO            Farm Labor Contractors and Crew Leaders   
11  Larimer County, CO                           Farm Management Services   

In [38]:
import numpy as np
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html, Input, Output

# Unique CZs for dropdown selection
unique_czs = df_around_wy["CZ20"].unique()

# Initialize Dash app
app = dash.Dash(__name__)

# Define app layout
app.layout = html.Div([
    html.H1("Interactive Treemap of CZ Industries", style={'textAlign': 'center'}),
    
    # Dropdown for CZ selection
    html.Label("Select a Commuting Zone:"),
    dcc.Dropdown(
        id="cz-dropdown",
        options=[{"label": f"CZ {cz}", "value": cz} for cz in unique_czs],
        value=unique_czs[0],
        clearable=False
    ),
    
    # Treemap visualization
    dcc.Graph(id="treemap"),
    
    # Industry details output
    html.H3("Industry Details:", style={'textAlign': 'center'}),
    dcc.Loading(
        id="loading-output",
        children=[html.Div(id="industry-details")],
        type="circle"
    )
])

# Callback to update treemap
@app.callback(
    Output("treemap", "figure"),
    Input("cz-dropdown", "value")
)
def update_treemap(selected_cz):
    df_czs = df_around_wy[df_around_wy["CZ20"] == selected_cz]
    if df_czs.empty:
        return px.treemap(title=f"No Data Available for CZ {selected_cz}")
    
    area_name = df_czs["Area Name"].mode()[0] if not df_czs["Area Name"].isna().all() else "Unknown Location"
    min_val = df_czs["Share_Deviation_Normalized"].min()
    max_val = df_czs["Share_Deviation_Normalized"].max()
    
    # Define color scale
    color_scale = [(0, "white"), (1, "darkgreen")]
    if min_val < 0:
        color_scale.insert(0, (0, "red"))
    
    fig = px.treemap(
        df_czs,
        path=["Industry Name"],
        values="Share_Deviation_Normalized",
        color="Share_Deviation_Normalized",
        color_continuous_scale=color_scale,
        color_continuous_midpoint=0,
        title=f"Treemap for {area_name} (Commuting Zone {selected_cz})",
        labels={"Industry Name": "Industry"}
    )
    
    return fig

# Callback to display industry details
@app.callback(
    Output("industry-details", "children"),
    Input("treemap", "clickData")
)
def display_industry_details(clickData):
    if not clickData:
        return html.Div("Click on an industry to view details.", style={'textAlign': 'center', 'fontSize': '18px', 'marginTop': '10px'})
    
    industry_name = clickData["points"][0]["label"]
    selected_row = df_around_wy[df_around_wy["Industry Name"] == industry_name]
    
    if selected_row.empty:
        return html.Div("No details available for this industry.", style={'textAlign': 'center', 'fontSize': '18px', 'marginTop': '10px'})
    
    selected_row = selected_row.iloc[0]  # Get the first matching row

    # Create a table-like layout for displaying details
    details_layout = html.Div([
        html.H4(f"Industry: {industry_name}", style={'textAlign': 'center', 'marginBottom': '10px'}),
        html.Table([
            html.Tbody([
                html.Tr([html.Td(html.B(col + ":"), style={'padding': '5px', 'width': '40%'}), html.Td(str(selected_row[col]), style={'padding': '5px'})])
                for col in selected_row.index
            ])
        ], style={'width': '80%', 'margin': 'auto', 'borderCollapse': 'collapse', 'border': '1px solid #ddd', 'padding': '10px'})
    ])

    return details_layout

# Run the Dash app
if __name__ == "__main__":
    app.run_server(debug=True, port=8051)


